In [1]:
import os

import numpy as np
from sklearn.linear_model import LogisticRegression

from utils import load_hidden_representations_from_hdf5, read_templates_from_file

----

In [2]:
# params
log_dir = "/logfiles"
model = "bigscience-T0"
module = "encoder"
task = "wic"

In [3]:
# assertions
assert model == "bigscience-T0"
assert module == "encoder"
assert task == "wic"

## Prompts

In [4]:
df = read_templates_from_file(f"/t0-analysis/prompts/{task}/all.csv")
# df = read_templates_from_file(f"/t0-analysis/prompts/{task}/fixed_prompt.csv")
# df = read_templates_from_file(f"/t0-analysis/prompts/{task}/fixed_target_yes_no.csv")
display(df)

,name,template,category,includes_targets,targets,target_ids,shuffle
0,gpt_3_without_targets,{sentence1} {sentence2} Question: Is the word ...,instructive,False,"▁No, ▁no, ▁Yes, ▁yes","0, 0, 1, 1",False
1,gpt_3_yes_no_with_targets,{sentence1} {sentence2} Question: Is the word ...,instructive,True,"▁No, ▁Yes","0, 1",False
2,affirmation_with_targets,Sentence A: {sentence1} Sentence B: {sentence2...,instructive,True,"▁Fal, ▁True","0, 1",False
3,grammar_homework_with_targets,"Decide whether the word ""{word}"" is used with ...",instructive,True,"▁No, ▁Yes","0, 1",False
4,polysemous_with_targets,"The word ""{word}"" has multiple meanings. Does ...",instructive,True,"▁No, ▁Yes","0, 1",False
5,question_context_without_targets,"Determine if the word ""{word}"" is used in the ...",instructive,False,"▁No, ▁no, ▁Yes, ▁yes","0, 0, 1, 1",False
6,question_meaning_without_targets,"Determine if the word ""{word}"" have the same m...",instructive,False,"▁No, ▁no, ▁Yes, ▁yes","0, 0, 1, 1",False
7,question_meaning_yes_no_with_targets,"Does the word ""{word}"" have the same meanining...",instructive,True,"▁No, ▁Yes","0, 1",False
8,same_sense_with_targets,Sentence 1: {sentence1} Sentence 2: {sentence2...,instructive,True,"▁No, ▁Yes","0, 1",False
9,similar_sense_without_targets,{sentence1} {sentence2} Similar sense of {word}?,instructive,False,"▁No, ▁no, ▁Yes, ▁yes","0, 0, 1, 1",False


## Train linear classifier

In [5]:
def unison_shuffled_copies(a, b):
    # from: https://stackoverflow.com/questions/4601373/better-way-to-shuffle-two-numpy-arrays-in-unison
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [7]:
use_pattern = [
    # "gpt_3_without_targets",
    "gpt_3_yes_no_with_targets",
    # "affirmation_with_targets",
    # "grammar_homework_with_targets",
    # "polysemous_with_targets",
    # "question_context_without_targets",
    # "question_meaning_without_targets",
    # "question_meaning_yes_no_with_targets",
    # "same_sense_with_targets",
    # "similar_sense_without_targets",
    # "similar_sense_yes_no_with_targets",
    "gpt_3_true_false_with_targets",
    # "gpt_3_cat_dog_with_targets",
]

In [8]:
# for layer in range(0, 10):
for layer in range(0, 25):
# for layer in range(24, 25):
    
    print("layer:", layer)
    file_names, prompt_names = [], []

    for _, row in df.iterrows():
        if row['name'] in use_pattern:
            file_names.append(f"{task}/{model}/{module}/{row['name']}/hidden_represenations_layer{layer}_avg.hdf5",)
            prompt_names.append(row['name'])

    # load hidden representations from hdf5 file
    representations = None
    classes = []
    n_sequences = 0

    for idx, file_name in enumerate(file_names):
        hidden_representations = load_hidden_representations_from_hdf5(os.path.join(log_dir, file_name))
        # print(hidden_representations.shape)
        n_sequences = hidden_representations.shape[0]

        if representations is None:
            representations = hidden_representations
        else:
            representations = np.concatenate((representations, hidden_representations), axis=0)

        classes += n_sequences * [idx] # assign representations to classes
    
    classes = np.asarray(classes)

    # shuffle representations and classes
    X, y = unison_shuffled_copies(representations, classes)
    print(X.shape, y.shape)

    # train linear classifier
    # multi_class='multinomial' uses a CE loss
    print('classifying between:', prompt_names)
    clf = LogisticRegression(random_state=0, max_iter=2000, multi_class='multinomial').fit(X, y)
    
    print(f'layer={layer}; accuracy on training data: ', clf.score(X, y))
    print('\n')



layer: 0


Reading embeddings: 100%|██████████| 638/638 [00:00<00:00, 5911.74it/s]


(1276, 4096) (1276,)
classifying between: ['gpt_3_yes_no_with_targets', 'gpt_3_true_false_with_targets']
layer=0; accuracy on training data:  1.0


layer: 1


Reading embeddings: 100%|██████████| 638/638 [00:00<00:00, 5918.39it/s]


(1276, 4096) (1276,)
classifying between: ['gpt_3_yes_no_with_targets', 'gpt_3_true_false_with_targets']
layer=1; accuracy on training data:  1.0


layer: 2


...

(1276, 4096) (1276,)
classifying between: ['gpt_3_yes_no_with_targets', 'gpt_3_true_false_with_targets']
layer=22; accuracy on training data:  1.0


layer: 23


Reading embeddings: 100%|██████████| 638/638 [00:00<00:00, 6028.47it/s]


(1276, 4096) (1276,)
classifying between: ['gpt_3_yes_no_with_targets', 'gpt_3_true_false_with_targets']
layer=23; accuracy on training data:  1.0


layer: 24


Reading embeddings: 100%|██████████| 638/638 [00:00<00:00, 6072.52it/s]


(1276, 4096) (1276,)
classifying between: ['gpt_3_yes_no_with_targets', 'gpt_3_true_false_with_targets']
layer=24; accuracy on training data:  0.9992163009404389


